We've tried to built a deep neural network for our problem. 
The results are really bad for the moment ...

In [1]:
from jyquickhelper import add_notebook_menu
add_notebook_menu()

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import tensorflow as tf
import pandas as pd
import numpy as np

import random

## Data

We do the same than our previous notebook called "A bad Simple Linear Regression"

In [4]:
from transplant.tools.dataset import Dataset 

dataset = Dataset()

X_train, X_test , y_train, y_test , X_col = dataset.get_data_merged_dynamic_flatten("One_Hot",True)
X_train_not_use, X_test_not_use , y_train_cls, y_test_cls , X_col_not_use = dataset.get_data_merged_dynamic_flatten()

Static merged with Flattent Dynamic (i.e. : took the mean for each time serie).
You can chose between One_Hot encoding ([0,1] , [1,0]) with 'One_Hot' for target data or not with 'cls'.
You can chose to center and reduce your data with the train set data with 'True'
Static merged with Flattent Dynamic (i.e. : took the mean for each time serie).
You can chose between One_Hot encoding ([0,1] , [1,0]) with 'One_Hot' for target data or not with 'cls'.
You can chose to center and reduce your data with the train set data with 'True'


In [5]:
from datetime import datetime


now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs"
logdir = "{}/run-{}/".format(root_logdir, now)

## Construction phase

In [6]:
n_inputs = X_train.shape[1]  
n_hidden1 = 10
n_hidden2 = 100
n_outputs = 2

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int32, shape=(None), name="y")

def neuron_layer(X, n_neurons, name, activation=None):
    with tf.name_scope(name):
        n_inputs = int(X.shape[1])
        stddev = 2 / np.sqrt(n_inputs)
        init = tf.truncated_normal((n_inputs, n_neurons), stddev=stddev)
        W = tf.Variable(init, name="kernel")
        b = tf.Variable(tf.zeros([n_neurons]), name="bias")
        Z = tf.matmul(X, W) + b
        if activation is not None:
            return activation(Z)
        else:
            return Z

with tf.name_scope("dnn"):
    hidden1 = neuron_layer(X, n_hidden1, name="hidden1",
                           activation=tf.nn.relu)
    hidden2 = neuron_layer(hidden1, n_hidden2, name="hidden2",
                           activation=tf.nn.relu)
    logits = neuron_layer(hidden2, n_outputs, name="outputs")

with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    
    loss = tf.reduce_mean(xentropy, name="loss")

learning_rate = 0.01

with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

init = tf.global_variables_initializer()
saver = tf.train.Saver()


## Run

In [7]:
def shuffle_batch(X, y, batch_size):
    rnd_idx = np.random.permutation(len(X))
    n_batches = len(X) // batch_size
    for batch_idx in np.array_split(rnd_idx, n_batches):
        X_batch, y_batch = X[batch_idx], y[batch_idx]
        yield X_batch, y_batch

In [8]:
n_epochs = 1000
batch_size = 50

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for X_batch, y_batch in shuffle_batch(X_train, y_train_cls, batch_size):
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        acc_batch = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_valid = accuracy.eval(feed_dict={X: X_test, y: y_test_cls})
        if epoch%100==0 :
            print(epoch, "Batch accuracy:", acc_batch, "Validation accuracy:", acc_valid)


0 Batch accuracy: 0.52 Validation accuracy: 0.5445545
100 Batch accuracy: 0.82 Validation accuracy: 0.5346535
200 Batch accuracy: 0.94 Validation accuracy: 0.55445546
300 Batch accuracy: 1.0 Validation accuracy: 0.55445546
400 Batch accuracy: 1.0 Validation accuracy: 0.56435645
500 Batch accuracy: 1.0 Validation accuracy: 0.56435645
600 Batch accuracy: 1.0 Validation accuracy: 0.55445546
700 Batch accuracy: 1.0 Validation accuracy: 0.56435645
800 Batch accuracy: 0.98 Validation accuracy: 0.55445546
900 Batch accuracy: 1.0 Validation accuracy: 0.56435645
